In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from metal.mmtl.trainer import MultitaskTrainer
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.slicing.slice_model import SliceModel

In [3]:
SEED = 1

### Initialize normal payloads

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 16},
    "freeze_bert":False,
    "bert_model": 'bert-large-uncased',
    "max_len": 128,
    "attention": False,
    "dropout": 0.1,
}
task_names = ["RTE"]

In [5]:
%%time

# Create tasks and payloads
tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Using random seed: 732000
Loading RTE Dataset



CPU times: user 25.7 s, sys: 2.64 s, total: 28.4 s
Wall time: 29.2 s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.00)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=test)])

### Initialize and train baseline model 

In [7]:
model = MetalModel(tasks, seed=SEED, verbose=False)

In [8]:
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads

from metal.mmtl.metal_model import MetalModel

SEED = 123
prev_model = MetalModel(tasks, seed=SEED, verbose=False)

import os
import torch
model_dir = '/dfs/scratch0/chami/metal/logs/2019_03_19/RTE_16_44_10'
model_path = os.path.join(model_dir, 'best_model.pth')
prev_model.load_weights(model_path)

Your destination state dict has different keys for the update key.


In [10]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    prev_model,
    payloads,
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=1e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=5,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=True,
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 156 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_27/13_59_39/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=7.23e-01, RTE_valid/RTE_gold/accuracy=8.77e-01] model:[train/all/loss=7.23e-01, train/all/lr=1.00e-05, valid/all/loss=6.11e-01]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.877
[0.21 epo]: RTE:[RTE_train/RTE_gold/loss=5.44e-01, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=5.44e-01, train/all/lr=1.00e-05, valid/all/loss=4.85e-01]
Saving model at iteration 0.21 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.953
[0.31 epo]: RTE:[RTE_train/RTE_gold/loss=4.11e-01, RTE_valid/RTE_gold/accuracy=9.46e-01] model:[train/all/loss=4.11e-01, train/all/lr=1.00e-05, valid/all/loss=3.85e-01]
[0.41 epo]: RTE:[RTE_train/RTE_gold/loss=3.16e-01, RTE_valid/RTE_gold/accuracy=9.49e-01] model:[train/all/loss=3.16e-01, train/all/lr=1.00e-05, valid/all/loss=3.08e-01]
[0.51 epo]: RTE:[RTE_train/RTE_gold/loss=2.71e-01, RTE_valid/RTE_gold/accuracy=9.49e-01] model:[train/all/loss=2.71e-01, train/all/lr=1.00e-05

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=1.29e-01, RTE_valid/RTE_gold/accuracy=9.57e-01] model:[train/all/loss=1.29e-01, train/all/lr=1.00e-05, valid/all/loss=1.42e-01]
Saving model at iteration 1.03 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.957
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=1.05e-01, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=1.05e-01, train/all/lr=1.00e-05, valid/all/loss=1.39e-01]
[1.23 epo]: RTE:[RTE_train/RTE_gold/loss=6.16e-02, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=6.16e-02, train/all/lr=1.00e-05, valid/all/loss=1.34e-01]
[1.33 epo]: RTE:[RTE_train/RTE_gold/loss=1.01e-01, RTE_valid/RTE_gold/accuracy=9.60e-01] model:[train/all/loss=1.01e-01, train/all/lr=1.00e-05, valid/all/loss=1.33e-01]
Saving model at iteration 1.33 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.960
[1.44 epo]: RTE:[RTE_train/RTE_gold/loss=4.28e-02, RTE_valid/RTE_gold/accuracy=9.57e-01] model:[train/all/loss=4.28e-02, train/all/lr=1.00e-05

[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=7.25e-02, RTE_valid/RTE_gold/accuracy=9.57e-01] model:[train/all/loss=7.25e-02, train/all/lr=1.00e-05, valid/all/loss=1.50e-01]
[2.15 epo]: RTE:[RTE_train/RTE_gold/loss=3.08e-02, RTE_valid/RTE_gold/accuracy=9.46e-01] model:[train/all/loss=3.08e-02, train/all/lr=1.00e-05, valid/all/loss=1.70e-01]
[2.26 epo]: RTE:[RTE_train/RTE_gold/loss=9.90e-02, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=9.90e-02, train/all/lr=1.00e-05, valid/all/loss=1.62e-01]
[2.36 epo]: RTE:[RTE_train/RTE_gold/loss=5.73e-02, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=5.73e-02, train/all/lr=1.00e-05, valid/all/loss=1.56e-01]
[2.46 epo]: RTE:[RTE_train/RTE_gold/loss=4.79e-02, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=4.79e-02, train/all/lr=1.00e-05, valid/all/loss=1.64e-01]
[2.56 epo]: RTE:[RTE_train/RTE_gold/loss=6.92e-02, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=6.92e-02, train/all/lr=1.00e-05, valid/all/l

[3.08 epo]: RTE:[RTE_train/RTE_gold/loss=7.04e-03, RTE_valid/RTE_gold/accuracy=9.60e-01] model:[train/all/loss=7.04e-03, train/all/lr=1.00e-05, valid/all/loss=1.50e-01]
[3.18 epo]: RTE:[RTE_train/RTE_gold/loss=5.24e-02, RTE_valid/RTE_gold/accuracy=9.53e-01] model:[train/all/loss=5.24e-02, train/all/lr=1.00e-05, valid/all/loss=1.67e-01]
[3.28 epo]: RTE:[RTE_train/RTE_gold/loss=7.86e-02, RTE_valid/RTE_gold/accuracy=9.60e-01] model:[train/all/loss=7.86e-02, train/all/lr=1.00e-05, valid/all/loss=1.52e-01]
[3.38 epo]: RTE:[RTE_train/RTE_gold/loss=3.88e-02, RTE_valid/RTE_gold/accuracy=9.60e-01] model:[train/all/loss=3.88e-02, train/all/lr=1.00e-05, valid/all/loss=1.53e-01]
[3.49 epo]: RTE:[RTE_train/RTE_gold/loss=8.45e-02, RTE_valid/RTE_gold/accuracy=9.60e-01] model:[train/all/loss=8.45e-02, train/all/lr=1.00e-05, valid/all/loss=1.52e-01]
[3.59 epo]: RTE:[RTE_train/RTE_gold/loss=3.10e-02, RTE_valid/RTE_gold/accuracy=9.60e-01] model:[train/all/loss=3.10e-02, train/all/lr=1.00e-05, valid/all/l

[4.10 epo]: RTE:[RTE_train/RTE_gold/loss=2.13e-02, RTE_valid/RTE_gold/accuracy=9.57e-01] model:[train/all/loss=2.13e-02, train/all/lr=1.00e-05, valid/all/loss=1.70e-01]
[4.21 epo]: RTE:[RTE_train/RTE_gold/loss=8.28e-02, RTE_valid/RTE_gold/accuracy=9.68e-01] model:[train/all/loss=8.28e-02, train/all/lr=1.00e-05, valid/all/loss=1.57e-01]
Saving model at iteration 4.21 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.968
[4.31 epo]: RTE:[RTE_train/RTE_gold/loss=2.25e-02, RTE_valid/RTE_gold/accuracy=9.68e-01] model:[train/all/loss=2.25e-02, train/all/lr=1.00e-05, valid/all/loss=1.57e-01]
[4.41 epo]: RTE:[RTE_train/RTE_gold/loss=6.96e-02, RTE_valid/RTE_gold/accuracy=9.64e-01] model:[train/all/loss=6.96e-02, train/all/lr=1.00e-05, valid/all/loss=1.54e-01]
[4.51 epo]: RTE:[RTE_train/RTE_gold/loss=1.18e-01, RTE_valid/RTE_gold/accuracy=9.64e-01] model:[train/all/loss=1.18e-01, train/all/lr=1.00e-05, valid/all/loss=1.58e-01]
[4.62 epo]: RTE:[RTE_train/RTE_gold/loss=8.12e-02, RTE_valid/RTE

/dfs/scratch0/vschen/metal-mmtl/metal/mmtl/trainer.py:915: UserWarning: MeTaL does not support calculating loss on the test set.
  warnings.warn(msg)


{'RTE/RTE_test/RTE_gold/accuracy': 0.0}
Cleaning checkpoints
Writing metrics to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_27/13_59_39/metrics.json
Writing log to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_27/13_59_39/log.json
Full model saved at /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_27/13_59_39/model.pkl
CPU times: user 15min 9s, sys: 3min 6s, total: 18min 15s
Wall time: 19min 26s


In [11]:
assert False

AssertionError: 

### Initialize slice payloads

In [12]:
# Create tasks and payloads
task_kwargs.update({"slice_dict": {
    "RTE": ["has_date", "BASE"]}
})
task_kwargs['attention'] = None

tasks_slice, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 773630
Loading RTE Dataset



Added label_set with 2490/2490 labels for task RTE_slice:has_date:ind to payload RTE_train.
Added label_set with 1383/2490 labels for task RTE_slice:has_date:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:ind to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:pred to payload RTE_train.
Added label_set with 277/277 labels for task RTE_slice:has_date:ind to payload RTE_valid.
Added label_set with 148/277 labels for task RTE_slice:has_date:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:BASE:ind to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:BASE:pred to payload RTE_valid.
Added label_set with 3000/3000 labels for task RTE_slice:has_date:ind to payload RTE_test.
Added label_set with 1660/3000 labels for task RTE_slice:has_date:pred to payload RTE_test.
Added label_set with 3000/3000 labels for task RTE_slice:BASE:ind to payload RTE_test.
Added label_s

### Evaluate baseline slices

In [13]:
tasks_slice, payloads_slice

([ClassificationTask(name=RTE, loss_multiplier=1.00),
  ClassificationTask(name=RTE_slice:has_date:ind, loss_multiplier=0.25),
  ClassificationTask(name=RTE_slice:has_date:pred, loss_multiplier=0.25),
  ClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.25),
  ClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.25)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:has_date:ind': 'RTE_slice:has_date:ind', 'RTE_slice:has_date:pred': 'RTE_slice:has_date:pred', 'RTE_slice:BASE:ind': 'RTE_slice:BASE:ind', 'RTE_slice:BASE:pred': 'RTE_slice:BASE:pred'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:has_date:ind': 'RTE_slice:has_date:ind', 'RTE_slice:has_date:pred': 'RTE_slice:has_date:pred', 'RTE_slice:BASE:ind': 'RTE_slice:BASE:ind', 'RTE_slice:BASE:pred': 'RTE_slice:BASE:pred'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:has_date:ind': 'RTE_slice:has_date:ind', 'RTE_slice

In [14]:
import copy
eval_payload = copy.deepcopy(payloads_slice[1])

# NOTE: we need to retarget slices to the original RTE head
# for label_name in ['RTE_slice:dash_semicolon', 'RTE_slice:more_people', 'RTE_slice:BASE']:
for label_name in ['RTE_slice:has_date', 'RTE_slice:BASE']:
    for slice_head_type in ['ind', 'pred']:
        eval_payload.retarget_labelset(f'{label_name}:{slice_head_type}', 'RTE')

labelset 'RTE_slice:has_date:ind' -> task 'RTE' (originally, RTE_slice:has_date:ind).
labelset 'RTE_slice:has_date:pred' -> task 'RTE' (originally, RTE_slice:has_date:pred).
labelset 'RTE_slice:BASE:ind' -> task 'RTE' (originally, RTE_slice:BASE:ind).
labelset 'RTE_slice:BASE:pred' -> task 'RTE' (originally, RTE_slice:BASE:pred).


In [15]:
eval_payload

Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:has_date:ind': 'RTE', 'RTE_slice:has_date:pred': 'RTE', 'RTE_slice:BASE:ind': 'RTE', 'RTE_slice:BASE:pred': 'RTE'}], split=valid)

In [ ]:
model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.9675090252707581,
 'RTE/RTE_valid/RTE_slice:has_date:ind/accuracy': 0.5342960288808665,
 'RTE/RTE_valid/RTE_slice:has_date:pred/accuracy': 0.9594594594594594,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.5451263537906137,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.9675090252707581}

### Initialize and train slice model

In [ ]:
from metal.mmtl.slicing.tasks import convert_to_slicing_tasks
slicing_tasks = convert_to_slicing_tasks(tasks_slice)
slicing_tasks

Modifying RTE out_features from 2 -> 1
Modifying RTE_slice:has_date:ind out_features from 2 -> 1
Modifying RTE_slice:has_date:pred out_features from 2 -> 1
Modifying RTE_slice:BASE:ind out_features from 2 -> 1
Modifying RTE_slice:BASE:pred out_features from 2 -> 1


[BinaryClassificationTask(name=RTE, loss_multiplier=1.00, slice_head_type=None),
 BinaryClassificationTask(name=RTE_slice:has_date:ind, loss_multiplier=0.25, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:has_date:pred, loss_multiplier=0.25, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.25, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.25, slice_head_type=pred)]

In [ ]:
slice_model = SliceModel(slicing_tasks, seed=SEED, verbose=False)

In [ ]:
# load weights from the base model
base_state_dict = model.state_dict()
slice_state_dict = slice_model.state_dict()
del base_state_dict['head_modules.RTE.module.module.weight']
del base_state_dict['head_modules.RTE.module.module.bias']
slice_state_dict.update(base_state_dict)
slice_model.load_state_dict(slice_state_dict)

In [ ]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    slice_model,
    payloads_slice,
#     task_metrics=[
#         "RTE/RTE_train/RTE_gold/loss", 
#         "RTE/RTE_train/RTE_slice:BASE:ind/loss",
#         "RTE/RTE_train/RTE_slice:dash_semicolon:ind/loss", 
#         "RTE/RTE_train/RTE_slice:more_people:ind/loss",
#         "RTE/RTE_train/RTE_slice:BASE:pred/loss",
#         "RTE/RTE_train/RTE_slice:dash_semicolon:pred/loss", 
#         "RTE/RTE_train/RTE_slice:more_people:pred/loss",
        
#         "RTE/RTE_valid/RTE_gold/accuracy",
#         "RTE/RTE_valid/RTE_slice:BASE:ind/accuracy", 
#         "RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy", 
#         "RTE/RTE_valid/RTE_slice:more_people:ind/accuracy",
#         "RTE/RTE_valid/RTE_slice:BASE:pred/accuracy", 
#         "RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy", 
#         "RTE/RTE_valid/RTE_slice:more_people:pred/accuracy",
#     ],
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=1e-5,
#     l2=1e-3,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=50,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=True
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 156 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_04_27/14_25_25/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=6.96e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:has_date:ind:[RTE_train/RTE_slice:has_date:ind/loss=1.74e-01, RTE_valid/RTE_slice:has_date:ind/accuracy=5.49e-01] RTE_slice:has_date:pred:[RTE_train/RTE_slice:has_date:pred/loss=2.18e-01, RTE_valid/RTE_slice:has_date:pred/accuracy=4.32e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.34e-01, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.25e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=9.06e-01] model:[train/all/loss=2.75e-01, train/all/lr=1.00e-05, valid/all/loss=2.62e-01]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.473
[0.21 epo]: RTE:[RTE_train/RTE_gold/loss=6.95e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:has_date:ind:[RTE_train/RTE_slice:has_date:ind/loss=1.72e-01, RTE_valid/RTE_slice:has_date:ind/accuracy=5.34e-01] RTE_slice:has_date:pred:[RTE_train/RTE_slice:has_

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:has_date:ind:[RTE_train/RTE_slice:has_date:ind/loss=1.64e-01, RTE_valid/RTE_slice:has_date:ind/accuracy=5.52e-01] RTE_slice:has_date:pred:[RTE_train/RTE_slice:has_date:pred/loss=4.31e-02, RTE_valid/RTE_slice:has_date:pred/accuracy=9.66e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.98e-02, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=4.62e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=9.53e-01] model:[train/all/loss=2.06e-01, train/all/lr=1.00e-05, valid/all/loss=2.09e-01]
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] RTE_slice:has_date:ind:[RTE_train/RTE_slice:has_date:ind/loss=1.63e-01, RTE_valid/RTE_slice:has_date:ind/accuracy=5.52e-01] RTE_slice:has_date:pred:[RTE_train/RTE_slice:has_date:pred/loss=3.84e-02, RTE_valid/RTE_slice:has_date:pred/accuracy=9.53e-01] RTE_slice:BAS

#### Did we improve?

In [23]:
%%time
slice_model.score(payloads_slice[1])

CPU times: user 3.42 s, sys: 760 ms, total: 4.18 s
Wall time: 4.14 s


{'RTE/RTE_valid/RTE_gold/accuracy': 0.9169675090252708,
 'RTE_slice:has_date:ind/RTE_valid/RTE_slice:has_date:ind/accuracy': 0.924187725631769,
 'RTE_slice:has_date:pred/RTE_valid/RTE_slice:has_date:pred/accuracy': 0.9324324324324325,
 'RTE_slice:BASE:ind/RTE_valid/RTE_slice:BASE:ind/accuracy': 1.0,
 'RTE_slice:BASE:pred/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.9169675090252708}

In [24]:
slice_model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.9169675090252708,
 'RTE/RTE_valid/RTE_slice:has_date:ind/accuracy': 0.5342960288808665,
 'RTE/RTE_valid/RTE_slice:has_date:pred/accuracy': 0.9324324324324325,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.5306859205776173,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.9169675090252708}